#Import Libraries & Packages

In [135]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

pd.pandas.set_option('display.max_columns',None)


# Import Processed Data
## filename='data_processed1.csv

In [136]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [137]:
final_df=pd.read_csv("/content/drive/MyDrive/NFT Price Prediction/dataset/data_processed1.csv")
test=pd.read_csv('/content/drive/MyDrive/NFT Price Prediction/dataset/nfts_predict.csv')

In [138]:
final_df.head(5)

,Unnamed: 0,global_index,nft_id,collection_id,rarity_score,openrarity_score,last_sale_date,last_sale_price,n_tweets_in_range,avg_likes,avg_replies,avg_retweets,min_likes,min_replies,min_retweets,max_likes,max_replies,max_retweets,total_supply,creation_date,verification_status,n_of_traits,contract_type,seller_fees,openrarity_enabled,has_discord,has_medium,sale_year,sale_month,creation_to_last_sale,openrarity_rankRare_var,15000.0,15555.0,5555.0,8849.0,9928.0,9999.0,nan,openrarity_max_rankRare_var
0,0,-2.432025,-6.999358,0.813622,-2.940693,-0.561648,0.122459,1.266732,0.947537,1.439493,0.380163,1.201866,-0.747717,-0.640385,0.082130,1.398811,0.446277,1.400544,-12.259114,1.071204,-1.089063,-2.375507,0.880729,1.037420,-0.565829,0.820615,-0.326514,0.342080,0.958217,0.422459,-0.565829,-0.177495,-0.180854,-0.152135,-0.13576,-0.143638,-0.206027,0.565829,-0.198551
1,1,-2.026188,-6.999358,0.728050,-0.468541,-0.561648,-2.047277,1.200357,0.208027,1.235307,0.705850,1.144705,-0.747717,-0.640385,0.907442,1.199975,0.747738,1.180244,-0.607367,0.281865,-1.089063,-1.070886,0.880729,-0.424798,-0.565829,0.820615,-0.326514,-2.923295,0.363493,-0.567870,-0.565829,-0.177495,-0.180854,-0.152135,-0.13576,-0.143638,-0.206027,0.565829,-0.198551
2,2,-2.026157,-6.428333,0.728050,-0.213949,-0.561648,1.086786,3.180572,0.208027,1.235307,0.705850,1.144705,-0.747717,-0.640385,0.907442,1.199975,0.747738,1.180244,-0.607367,0.281865,-1.089063,-1.070886,0.880729,-0.424798,-0.565829,0.820615,-0.326514,0.342080,0.363493,0.741303,-0.565829,-0.177495,-0.180854,-0.152135,-0.13576,-0.143638,-0.206027,0.565829,-0.198551
3,3,-2.026125,-6.094305,0.728050,-0.275608,-0.561648,-2.047277,3.502910,0.208027,1.235307,0.705850,1.144705,-0.747717,-0.640385,0.907442,1.199975,0.747738,1.180244,-0.607367,0.281865,-1.089063,-1.070886,0.880729,-0.424798,-0.565829,0.820615,-0.326514,-2.923295,0.363493,-0.567870,-0.565829,-0.177495,-0.180854,-0.152135,-0.13576,-0.143638,-0.206027,0.565829,-0.198551
4,4,-2.026094,-5.857308,0.728050,-0.366154,-0.561648,0.845705,4.009567,0.208027,1.235307,0.705850,1.144705,-0.747717,-0.640385,0.907442,1.199975,0.747738,1.180244,-0.607367,0.281865,-1.089063,-1.070886,0.880729,-0.424798,-0.565829,0.820615,-0.326514,0.342080,0.581304,0.780392,-0.565829,-0.177495,-0.180854,-0.152135,-0.13576,-0.143638,-0.206027,0.565829,-0.198551


In [139]:
final_df.drop(columns=['Unnamed: 0'],inplace=True)

In [140]:
df_Train=final_df.iloc[:final_df.shape[0]-test.shape[0],:]
df_Test=final_df.iloc[final_df.shape[0]-test.shape[0]:,:]

In [141]:
(df_Train.shape,df_Test.shape)

((309910, 38), (181214, 38))

In [142]:
df_Test.drop(['last_sale_price'],axis=1,inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [143]:
X_train=df_Train.drop(['last_sale_price'],axis=1)
y_train=df_Train['last_sale_price']

X_full=X_train.copy()
y_full=y_train.copy()
    
X_train,x_valid,y_train,y_valid = train_test_split(X_train,y_train,test_size = 0.1,random_state=1)

# Modeling

In [144]:
reg_ols= sm.OLS(np.log(y_train), X_train)
results = reg_ols.fit()
results.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                       Results: Ordinary least squares
=============================================================================
Model:                  OLS                Adj. R-squared:       0.540       
Dependent Variable:     last_sale_price    AIC:                  1175839.1980
Date:                   2023-02-27 05:49   BIC:                  1176208.0516
No. Observations:       278919             Log-Likelihood:       -5.8788e+05 
Df Model:               34                 F-statistic:          9621.       
Df Residuals:           278884             Prob (F-statistic):   0.00        
R-squared:              0.540              Scale:                3.9658      
-----------------------------------------------------------------------------
                             Coef.  Std.Err.     t     P>|t|   [0.025  0.975]
-----------------------------------------------------------------------------
global_index                -0.3882   0.0137  -28.3639 0.0000 -0.4151 -0.3614
nft_id                      -0.0363   0.0046   -7.8229 0.0000 -0.0454 -0.0272
collection_id               -0.0997   0.0166   -6.0142 0.0000 -0.1322 -0.0672
rarity_score                -0.0644   0.0127   -5.0744 0.0000 -0.0892 -0.0395
openrarity_score             0.0879   0.0332    2.6489 0.0081  0.0229  0.1529
last_sale_date               0.1070   0.0087   12.2997 0.0000  0.0899  0.1240
n_tweets_in_range            0.2351   0.0189   12.4648 0.0000  0.1982  0.2721
avg_likes                    0.7062   0.0890    7.9342 0.0000  0.5318  0.8807
avg_replies                 -3.6829   0.0637  -57.7966 0.0000 -3.8078 -3.5580
avg_retweets                 1.5140   0.0829   18.2606 0.0000  1.3515  1.6765
min_likes                   -1.8271   0.0253  -72.2923 0.0000 -1.8767 -1.7776
min_replies                  3.6657   0.0388   94.4365 0.0000  3.5896  3.7418
min_retweets                -0.2558   0.0310   -8.2536 0.0000 -0.3165 -0.1950
max_likes                    1.5423   0.0897   17.1859 0.0000  1.3664  1.7182
max_replies                  0.5184   0.0471   11.0002 0.0000  0.4261  0.6108
max_retweets                 0.1196   0.0827    1.4470 0.1479 -0.0424  0.2816
total_supply                 1.0957   0.0184   59.5056 0.0000  1.0596  1.1318
creation_date               -0.5236   0.0128  -40.9455 0.0000 -0.5487 -0.4986
verification_status          0.3857   0.0269   14.3487 0.0000  0.3331  0.4384
n_of_traits                 -0.5577   0.0157  -35.5898 0.0000 -0.5884 -0.5270
contract_type                1.3661   0.0229   59.5828 0.0000  1.3211  1.4110
seller_fees                 -1.0932   0.0108 -101.2841 0.0000 -1.1143 -1.0720
openrarity_enabled          -0.7278   0.0164  -44.3023 0.0000 -0.7600 -0.6956
has_discord                  0.2664   0.0230   11.5950 0.0000  0.2214  0.3115
has_medium                   0.0670   0.0184    3.6453 0.0003  0.0310  0.1030
sale_year                    0.0958   0.0089   10.8007 0.0000  0.0784  0.1132
sale_month                  -0.1105   0.0068  -16.2909 0.0000 -0.1238 -0.0972
creation_to_last_sale       -0.2241   0.0066  -34.0146 0.0000 -0.2370 -0.2112
openrarity_rankRare_var     -0.7278   0.0164  -44.3023 0.0000 -0.7600 -0.6956
15000.0                      0.1386   0.0201    6.8797 0.0000  0.0991  0.1780
15555.0                      1.0122   0.0199   50.9270 0.0000  0.9733  1.0512
5555.0                       0.8749   0.0159   54.8654 0.0000  0.8436  0.9061
8849.0                       0.5507   0.0134   41.1030 0.0000  0.5244  0.5770
9928.0                       0.4912   0.0179   27.3822 0.0000  0.4560  0.5263
9999.0                      -0.2810   0.0121  -23.3158 0.0000 -0.3047 -0.2574
nan                          0.7278   0.0164   44.3023 0.0000  0.6956  0.7600
openrarity_max_rankRare_var 17.8692   0.0583  306.7602 0.0000 17.7551 17.9834
-----------------------------------------------------------------------------
Omnibus:              39032.890      Durbin-Watson:         1.999         

In [145]:
y_v_sm=np.exp(results.predict(x_valid))
y_t_sm=np.exp(results.predict(df_Test))

In [146]:
def rmse(predictions, targets):
    r= np.sqrt(((predictions - targets) ** 2).mean())
    return np.exp(-r/10 )

In [147]:
rmse(y_v_sm,y_valid)

0.3000327072267818

## Gradient Boosting

In [148]:

gbrt= GradientBoostingRegressor(max_depth=5, min_samples_split=200, n_estimators=50,
                          random_state=42)

# gbrt.fit(X_train, np.log(y_train))

In [149]:
# y_pred_test2=np.exp(gbrt.predict(df_Test))
    
# y_pred_valid2=np.exp(gbrt.predict(x_valid))
# rmse(y_pred_valid2,y_valid)

In [150]:
# y_pred_test2

### Hyper parameter Tuning

In [151]:
n_estimators = [40, 45,50, 55,60,80]
max_depth = [ 4, 5, 6]
learning_rate=[0.1,0.15,0.16,0.17,0.18,0.19,0.20,0.21,0.22,0.23]
min_samples_split=[100,125,150,200,250]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    # 'learning_rate':learning_rate,
    'min_samples_split':min_samples_split
    }

In [152]:
from sklearn.model_selection import RandomizedSearchCV 
random_cv_gbm = RandomizedSearchCV(estimator=gbrt,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=15,
            scoring = 'neg_root_mean_squared_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [153]:
# random_cv_grt.fit(X_train[:30000],np.log(y_train[:30000]))

In [154]:
# random_cv_grt.best_estimator_

## XGBoost

In [155]:
regressor=XGBRegressor(gpu_id=1, interaction_constraints='', max_depth=5,
             min_child_weight=3, monotone_constraints='(1,-1)', n_jobs=4,
             num_parallel_tree=1, tree_method='exact', validate_parameters=1,
             verbosity=3)
# regressor.fit(X_train,np.log(y_train))

In [156]:
# y_t_xg=np.exp(regressor.predict(df_Test))
    
# y_p_xg=np.exp(regressor.predict(x_valid))
# rmse(y_p_xg,y_valid)

In [157]:
# y_t_xg

### Hyper parameter Tuning

In [158]:
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]

In [159]:
## Hyper Parameter Optimization


n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 8,10, 12]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [160]:
from sklearn.metrics import SCORERS
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [161]:
random_cv_xgb = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=10,
            scoring = 'neg_root_mean_squared_error',n_jobs = 5,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [162]:
# random_cv.fit(X_train[:10000],np.log(y_train[:10000]))

In [163]:
# random_cv.best_estimator_

## Panel Regression

### Single Algorithm Based Panel Regression



In [164]:
def panel(df,alg,variable,error_func,pred_features,test):
  variable_val=np.array(list(df[variable].unique()))
  ID=df.copy()
  output=pd.DataFrame(columns=ID.columns)
  small_count=[]
  df_train=df.iloc[df.shape[0]-test.shape[0]:]
  for i in range(len(variable_val)):
    print(i)
    df_temp=df[df[variable]==variable_val[i]]
    df_train_temp=df_temp[df_temp[pred_features].isna()==False]
    df_test_temp=df_temp[df_temp[pred_features].isna()==True]
    # print(df_train_temp.shape,df_test_temp.shape)
    if df_test_temp.shape[0]<4 or df_train_temp.shape[0]<4:
      small_count.append(variable_val[i])
      continue
    else:
      df_test_temp=df_test_temp.drop(columns=[pred_features])
      x_train_temp=df_train_temp.drop(columns=[pred_features])
      y_train_temp=df_train_temp[pred_features]

      alg.fit(x_train_temp,y_train_temp)
      df_test_temp[pred_features]=alg.predict(df_test_temp)
      output=pd.concat([output,df_test_temp],axis='rows')


  ID=ID.drop(columns=[pred_features])
  final_out=output[[pred_features]]
  ID=pd.concat([ID,final_out],axis="columns")
  df2=ID.copy()
  df_train_temp=df2[df2[pred_features].isna()==False]
  df_test_temp=df2[df2[pred_features].isna()==True]
  df_test_temp=df_test_temp.drop(columns=[pred_features])
  x_train_temp=df_train_temp.drop(columns=[pred_features])
  y_train_temp=df_train_temp[pred_features]

  alg.fit(x_train_temp,y_train_temp)
  df_test_temp[pred_features]=alg.predict(df_test_temp)
  output=pd.concat([output,df_test_temp],axis='rows')

  ID=ID.drop(columns=[pred_features])

  ID=pd.concat([ID,output[[pred_features]]],axis="columns")
  # 
  return(ID[[pred_features]])


### Tuned Panel Regression

In [165]:
def panel_tune(df,alg,variable,error_func,pred_features,test,hyp):
  variable_val=np.array(list(df[variable].unique()))
  ID=df.copy()
  output=pd.DataFrame(columns=ID.columns)
  small_count=[]
  df_train=df.iloc[df.shape[0]-test.shape[0]:]
  for i in range(len(variable_val)):
    print(i)
    df_temp=df[df[variable]==variable_val[i]]
    df_train_temp=df_temp[df_temp[pred_features].isna()==False]
    df_test_temp=df_temp[df_temp[pred_features].isna()==True]
    # print(df_train_temp.shape,df_test_temp.shape)
    if df_test_temp.shape[0]<4 or df_train_temp.shape[0]<4:
      small_count.append(variable_val[i])
      continue
    else:
      df_test_temp=df_test_temp.drop(columns=[pred_features])
      x_train_temp=df_train_temp.drop(columns=[pred_features])
      y_train_temp=df_train_temp[pred_features]
      hyp.fit(x_train_temp,y_train_temp)
      best_est=hyp.best_estimator_
      print(best_est)
      best_est.fit(x_train_temp,y_train_temp)
      df_test_temp[pred_features]=best_est.predict(df_test_temp)
      output=pd.concat([output,df_test_temp],axis='rows')


  ID=ID.drop(columns=[pred_features])
  final_out=output[[pred_features]]
  ID=pd.concat([ID,final_out],axis="columns")
  df2=ID.copy()
  df_train_temp=df2[df2[pred_features].isna()==False]
  df_test_temp=df2[df2[pred_features].isna()==True]
  df_test_temp=df_test_temp.drop(columns=[pred_features])
  x_train_temp=df_train_temp.drop(columns=[pred_features])
  y_train_temp=df_train_temp[pred_features]

  hyp.fit(x_train_temp,y_train_temp)
  best_est=hyp.best_estimator_
  print(best_est)
  best_est.fit(x_train_temp,y_train_temp)
  df_test_temp[pred_features]=best_est.predict(df_test_temp)
  output=pd.concat([output,df_test_temp],axis='rows')

  ID=ID.drop(columns=[pred_features])

  ID=pd.concat([ID,output[[pred_features]]],axis="columns")
  # 
  return(ID[[pred_features]])


### Pre-Tuned Panel Regression

In [166]:
def panel_pre_tune(df,tuned,variable,error_func,pred_features,test):
  variable_val=np.array(list(df[variable].unique()))
  ID=df.copy()
  output=pd.DataFrame(columns=ID.columns)
  small_count=[]
  df_train=df.iloc[df.shape[0]-test.shape[0]:]
  for i in range(len(variable_val)):
    print(i)
    df_temp=df[df[variable]==variable_val[i]]
    # df_temp=df_temp.drop(columns=[variable])
    df_train_temp=df_temp[df_temp[pred_features].isna()==False]
    df_test_temp=df_temp[df_temp[pred_features].isna()==True]
    print(df_train_temp.shape,df_test_temp.shape)
    if df_test_temp.shape[0]<4 or df_train_temp.shape[0]<4:
      small_count.append(variable_val[i])
      continue
    else:
      df_test_temp=df_test_temp.drop(columns=[pred_features])
      x_train_temp=df_train_temp.drop(columns=[pred_features])
      y_train_temp=df_train_temp[pred_features]
      alg=tuned[i]
      print(alg)
      alg.fit(x_train_temp,y_train_temp)
      df_test_temp[pred_features]=alg.predict(df_test_temp)
      output=pd.concat([output,df_test_temp],axis='rows')

  ID=ID.drop(columns=[pred_features])
  final_out=output[[pred_features]]
  ID=pd.concat([ID,final_out],axis="columns")
  df2=ID.copy()
  df_train_temp=df2[df2[pred_features].isna()==False]
  df_test_temp=df2[df2[pred_features].isna()==True]
  print(df_train_temp.shape,df_test_temp.shape)
  df_test_temp=df_test_temp.drop(columns=[pred_features])
  x_train_temp=df_train_temp.drop(columns=[pred_features])
  y_train_temp=df_train_temp[pred_features]

  # hyp.fit(x_train_temp,y_train_temp)
  # best_est=hyp.best_estimator_
  # print(best_est)
  alg=tuned[24]
  print(alg)
  alg.fit(x_train_temp,y_train_temp)
  df_test_temp[pred_features]=alg.predict(df_test_temp)
  output=pd.concat([output,df_test_temp],axis='rows')

  ID=ID.drop(columns=[pred_features])

  ID=pd.concat([ID,output[[pred_features]]],axis="columns")
  # 
  return(ID[[pred_features]])


In [167]:

km = KMeans(n_clusters=25, random_state=1)
new =final_df.drop(columns=['last_sale_price'])

In [168]:
# visualizer = KElbowVisualizer(km, k=(20,30))
# visualizer.fit(new)        # Fit the data to the visualizer
# visualizer.show()

In [169]:
km.fit(np.array(new))
predict=km.predict(new)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


In [170]:
final_df['cluster']=predict

In [171]:
final_df['cluster'].value_counts()

9     48552
2     48257
22    32540
15    32267
17    29899
24    28575
0     24190
3     21377
23    20216
12    19602
1     18887
4     18627
13    16206
6     15555
20    15441
5     15000
11    11110
10    10000
7     10000
18     9999
19     9999
8      9928
21     9508
14     8888
16     6501
Name: cluster, dtype: int64

In [172]:
new=final_df.copy()
new['last_sale_price']=np.log(new['last_sale_price'])

### Fit

#### Solution 1

In [173]:
panel_test=panel(new,gbrt,'cluster',rmse,'last_sale_price',test)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [174]:
S1=panel_test[final_df.shape[0]-test.shape[0]:]
np.exp(S1)

,last_sale_price
309910,1.087091
309911,0.222507
309912,3.253762
309913,1.087091
309914,0.741355
...,...
491119,0.015961
491120,0.016269
491121,0.021178
491122,0.021178


In [175]:
S1['last_sale_price']=np.exp(S1['last_sale_price'])

<ipython-input-175-ab0e51b06980>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  S1['last_sale_price']=np.exp(S1['last_sale_price'])


#### Solution 2

In [176]:
# panel_test=panel_tune(new,gbrt,'cluster',rmse,'last_sale_price',test,random_cv_grt)
# S2=panel_test[final_df.shape[0]-test.shape[0]:]
# S2['last_sale_price']=np.exp(S2['last_sale_price'])
# S2.head()

In [177]:
pre_tuned_gbm={1:GradientBoostingRegressor(max_depth=4, min_samples_split=150, n_estimators=40,
                          random_state=42),
               3:GradientBoostingRegressor(max_depth=4, min_samples_split=250, n_estimators=50,
                          random_state=42),
               6:GradientBoostingRegressor(max_depth=6, min_samples_split=250, n_estimators=50,
                          random_state=42),
               7:GradientBoostingRegressor(max_depth=5, min_samples_split=250, n_estimators=50,
                          random_state=42),
               8: GradientBoostingRegressor(max_depth=4, min_samples_split=100, n_estimators=40,
                          random_state=42),
               11:GradientBoostingRegressor(max_depth=4, min_samples_split=100, n_estimators=40,
                          random_state=42),
               16: GradientBoostingRegressor(max_depth=4, min_samples_split=250, n_estimators=50,
                          random_state=42),
             24:GradientBoostingRegressor(max_depth=6, min_samples_split=250, n_estimators=50,
                          random_state=42)}

In [178]:
tuned_list=[pre_tuned_gbm]
hyp=[random_cv_gbm]
for x in range(len(tuned_list)):
  panel_temp=panel_pre_tune(new,tuned_list[x],'cluster',rmse,'last_sale_price',test)
  if x==0:
    stack=panel_temp[final_df.shape[0]-test.shape[0]:]
  else:
    out=panel_temp[final_df.shape[0]-test.shape[0]:]
    stack=pd.concat([stack,out],axis='columns')

0
(28575, 39) (0, 39)
1
(8888, 39) (9999, 39)
GradientBoostingRegressor(max_depth=4, min_samples_split=150, n_estimators=40,
                          random_state=42)
2
(9928, 39) (0, 39)
3
(5442, 39) (9999, 39)
GradientBoostingRegressor(max_depth=4, min_samples_split=250, n_estimators=50,
                          random_state=42)
4
(6501, 39) (0, 39)
5
(15555, 39) (0, 39)
6
(18241, 39) (3136, 39)
GradientBoostingRegressor(max_depth=6, min_samples_split=250, n_estimators=50,
                          random_state=42)
7
(24383, 39) (8157, 39)
GradientBoostingRegressor(max_depth=5, min_samples_split=250, n_estimators=50,
                          random_state=42)
8
(31141, 39) (1126, 39)
GradientBoostingRegressor(max_depth=4, min_samples_split=100, n_estimators=40,
                          random_state=42)
9
(19602, 39) (0, 39)
10
(11110, 39) (0, 39)
11
(24159, 39) (31, 39)
GradientBoostingRegressor(max_depth=4, min_samples_split=100, n_estimators=40,
                          random_

In [179]:
names=['GBM']
stack.columns=names
S2=np.exp(stack)
S2.head()

,GBM
309910,1.040449
309911,0.249687
309912,2.798193
309913,1.040449
309914,0.976794


#### Solution 3

In [180]:
pre_tuned_xgb={1:XGBRegressor(gpu_id=1, interaction_constraints='', max_depth=2,
             min_child_weight=4, monotone_constraints='(1,-1)', n_jobs=4,
             num_parallel_tree=1, tree_method='exact', validate_parameters=1,
             verbosity=3),
               3:XGBRegressor(base_score=0.25, booster='gblinear', gpu_id=1,
             interaction_constraints='', learning_rate=0.2,
             monotone_constraints='(1,-1)', n_jobs=4, num_parallel_tree=1,
             tree_method='exact', validate_parameters=1, verbosity=3),
               6:XGBRegressor(base_score=1, booster='gblinear', gpu_id=1,
             interaction_constraints='', learning_rate=0.2, max_depth=5,
             min_child_weight=3, monotone_constraints='(1,-1)',
             n_estimators=900, n_jobs=4, num_parallel_tree=1,
             tree_method='exact', validate_parameters=1, verbosity=3),
               7:XGBRegressor(base_score=0.25, booster='gblinear', gpu_id=1,
             interaction_constraints='', learning_rate=0.2,
             monotone_constraints='(1,-1)', n_jobs=4, num_parallel_tree=1,
             tree_method='exact', validate_parameters=1, verbosity=3),
               8: XGBRegressor(base_score=1, booster='gblinear', gpu_id=1,
             interaction_constraints='', learning_rate=0.2, max_depth=5,
             min_child_weight=3, monotone_constraints='(1,-1)',
             n_estimators=900, n_jobs=4, num_parallel_tree=1,
             tree_method='exact', validate_parameters=1, verbosity=3),
               11:XGBRegressor(booster='gblinear', gpu_id=1, interaction_constraints='',
             max_depth=8, min_child_weight=4, monotone_constraints='(1,-1)',
             n_estimators=1100, n_jobs=4, num_parallel_tree=1,
             tree_method='exact', validate_parameters=1, verbosity=3) ,
               16: XGBRegressor(base_score=1, booster='gblinear', gpu_id=1,
             interaction_constraints='', learning_rate=0.2, max_depth=5,
             min_child_weight=3, monotone_constraints='(1,-1)',
             n_estimators=900, n_jobs=4, num_parallel_tree=1,
             tree_method='exact', validate_parameters=1, verbosity=3),
             24:XGBRegressor(base_score=0.25, booster='gblinear', gpu_id=1,
             interaction_constraints='', learning_rate=0.2,
             monotone_constraints='(1,-1)', n_jobs=4, num_parallel_tree=1,
             tree_method='exact', validate_parameters=1, verbosity=3)}

In [181]:
# panel_test=panel_tune(new,regressor,'cluster',rmse,'last_sale_price',test,random_cv_xgb)
# S3=panel_test[final_df.shape[0]-test.shape[0]:]
# S3['last_sale_price']=np.exp(S3['last_sale_price'])
# S3.head()

In [182]:
tuned_list=[pre_tuned_xgb]
hyp=[random_cv_xgb]
for x in range(len(tuned_list)):
  panel_temp=panel_pre_tune(new,tuned_list[x],'cluster',rmse,'last_sale_price',test)
  if x==0:
    stack=panel_temp[final_df.shape[0]-test.shape[0]:]
  else:
    out=panel_temp[final_df.shape[0]-test.shape[0]:]
    stack=pd.concat([stack,out],axis='columns')

0
(28575, 39) (0, 39)
1
(8888, 39) (9999, 39)
XGBRegressor(gpu_id=1, interaction_constraints='', max_depth=2,
             min_child_weight=4, monotone_constraints='(1,-1)', n_jobs=4,
             num_parallel_tree=1, tree_method='exact', validate_parameters=1,
             verbosity=3)
[05:51:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:51:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[05:51:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[05:51:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[05:51:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[05:51:35] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning en

In [183]:
names=['XGB']
stack.columns=names
S3=np.exp(stack)
S3.head()

,XGB
309910,1.009332
309911,1.028892
309912,2.736051
309913,1.009332
309914,0.760658


# Final

In [184]:
df_result1=pd.DataFrame(S1['last_sale_price'].to_list(),columns=["last_sale_price"])
df_result2=pd.DataFrame(S2['GBM'].to_list(),columns=["last_sale_price"])
df_result3=pd.DataFrame(S3['XGB'].to_list(),columns=["last_sale_price"])

In [185]:
df_result1.head(5)

,last_sale_price
0,1.087091
1,0.222507
2,3.253762
3,1.087091
4,0.741355


In [186]:
df_result2.head(5)

,last_sale_price
0,1.040449
1,0.249687
2,2.798193
3,1.040449
4,0.976794


In [187]:
df_result3.head(5)

,last_sale_price
0,1.009332
1,1.028892
2,2.736051
3,1.009332
4,0.760658


In [188]:
s=pd.read_csv('/content/drive/MyDrive/NFT Price Prediction/dataset/submission_format.csv')

In [189]:
s.head()

,global_index,last_sale_price
0,0,0.234588
1,1,0.243322
2,2,0.231398
3,3,0.150778
4,4,2.015249


In [190]:
result1=pd.concat([s['global_index'],df_result1],axis='columns')
result2=pd.concat([s['global_index'],df_result2],axis='columns')
result3=pd.concat([s['global_index'],df_result3],axis='columns')

In [105]:
result1.to_csv('solution1.csv',index=False)
result2.to_csv('solution2.csv',index=False)
result3.to_csv('solution3.csv',index=False)